# Домашнее задание 3
### Андреев Никита

In [2]:
import sys
import time
import matplotlib.pyplot as plt
import pylab
import numpy as np
import scipy
import scipy.optimize as opt
import scipy.sparse
import sklearn.datasets

Функции оракулов:

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def oracle(w, X, labels, order=0, no_function=False):
    Xw = X.dot(w)
    sigmoids = [sigmoid(l * xw) for xw, l in zip(Xw, labels)]

    f = 0
    if not no_function:
        f = -1 / X.shape[0] * np.sum([np.log(s) for s in sigmoids])

    if order == 0:
        return f

    grad_coeffs = np.array([l * (1 - s) for s, l in zip(sigmoids, labels)])
    X1 = X.multiply(grad_coeffs.reshape(-1, 1))
    g = -1 / X.shape[0] * np.array(X1.sum(axis=0)).reshape(X.shape[1])

    if order == 1:
        return f, g, 0

    hess_coeffs = np.array([s * (1 - s) for s in sigmoids])
    h = lambda v: 1 / X.shape[0] * X.transpose().dot(X.dot(v) * hess_coeffs)

    if order == 2:
        return f, g, h

## В качестве данных впредь будем использовать три датасета:

a1a:

In [4]:
a1a = sklearn.datasets.load_svmlight_file('../data/a1a.txt')
X = a1a[0]
dummy = scipy.sparse.csr_matrix([[1] for i in range(X.shape[0])])
X_a1a = scipy.sparse.hstack([X, dummy])
labels_a1a = a1a[1]

breast cancer: (координаты нормированы от -1 до 1)

In [5]:
breast_cancer = sklearn.datasets.load_svmlight_file('../data/breast-cancer_scale.txt')
X = breast_cancer[0]
dummy = scipy.sparse.csr_matrix([[1] for i in range(X.shape[0])])
X_cancer = scipy.sparse.hstack([X, dummy])
labels_cancer = breast_cancer[1]-3

Случайный сгенерированный:

In [6]:
def random_dataset(alpha, beta):
    xs = np.random.normal(size=(1000, alpha.shape[0]))
    labels = np.array([np.sign(np.dot(alpha, x) + beta) for x in xs])
    return xs, labels


In [7]:
alpha = 2 * np.random.random(11) - 1
X, labels_rand = random_dataset(alpha[:-1], alpha[-1])
dummy = scipy.sparse.csr_matrix([[1] for i in range(X.shape[0])])
X_rand = scipy.sparse.hstack([X, dummy])

#### Методы из дз 2:

In [21]:
def golden_search_bounded(fun, a0, b0, eps=100*sys.float_info.epsilon, args=()):
    ratio = (1 + 5 ** 0.5) / 2

    def step(a, b, c, fc, onumber):
        if b - a < eps:
            return a, fun(a, *args), onumber + 1
        else:
            d = a + b - c
            fd = fun(d)
            if c > d:
                c, d = d, c
                fc, fd = fd, fc
            if fc < fd:
                return step(a, d, c, fc, onumber + 1)
            else:
                return step(c, b, d, fd, onumber + 1)

    c0 = a0 + (b0 - a0) / ratio
    solution = step(a0, b0, c0, fun(c0, *args), 0)
    return solution[0], solution[2]


def golden_search(fun, eps=100*sys.float_info.epsilon, args=()):
    a, _, b, _, _, _, onumber = opt.bracket(fun, args=args)
    if b < a:
        a, b = b, a
    gsb = golden_search_bounded(fun, a, b, eps=eps, args=args)
    return gsb[0], gsb[1] + onumber


def armiho(fun, c=0.1, k=10, f0=None, df0=None):
    x = 1
    oracle = 0

    if f0 == None:
        f0 = fun(0)
        oracle += 1

    if df0 is None:
        df0 = der(fun, 0)
        oracle += 2

    fx = fun(x)
    fkx = fun(k * x) 
    oracle += 2

    while True:
        if fx > f0 + c * df0 * x:
            x /= k
            fkx = fx
            fx = fun(x)
            oracle += 1
        elif fkx < f0 + k * c * df0 * x:
            x *= k
            fx = fkx
            fkx = fun(k*x)
            oracle += 1
        else:
            break

    return x, fx, oracle


def nester(fun, f0, d, L0=1):
    L = L0
    oracle = 0
    fx = fun(1 / L)
    oracle += 1
    
    while fx > f0 - 1 / (2 * L) * np.dot(d, d):
        L *= 2
        fx = fun(1 / L)
        oracle += 1
    

    return L, fx, oracle


Решение системы через разложение Холецкого:

In [13]:
def solveCholesky(h, d, eps=0.0001):
    G = np.array([h(x) for x in np.eye(d.shape[0])])
    if np.linalg.matrix_rank(G) < G.shape[0]:
        G = G + eps * np.eye(G.shape[0])
    L = np.linalg.cholesky(G)
    Ltx = scipy.linalg.solve_triangular(L, d, lower=True)
    return scipy.linalg.solve_triangular(np.transpose(L), Ltx, lower=False)

### Неточный метод сопряжённых градиентов:

In [ ]:
def solveCG(h, b, eta=0.01, policy='sqrtGradNorm'):
    b_norm = np.linalg.norm(b)
    if policy == 'sqrtGradNorm':
        pol = np.sqrt(b_norm)
    elif policy == 'gradNorm':
        pol = b_norm
    elif policy == 'const':
        pol = 1

    eps = eta * pol * b_norm
    x0 = np.random.random(b.shape[0]) * b_norm
    r = b - h(x0)
    p = r
    x = x0

    while True:
        rr = r.dot(r)
        hp = h(p)
        alpha = rr / np.dot(p, hp)
        x += alpha * p
        r -= alpha * hp
        print(np.linalg.norm(r))
        if np.linalg.norm(r) < eps:
            return x
        beta = r.dot(r) / rr
        p = r + beta * p

## Градиентный спуск и метод Ньютона:

Разница только в выборе направления - поэтому всё в одном методе

Параметры:

1. oracle - оракул
2. start - начальная точка
3. method - 'gradient descent' или 'newton'
4. __linear_solver - 'cg' или 'cholesky'__
5. __solver_kwargs - аргументы для linear_solver__
6. one_dim_search - способ выбора шага:
    'unit step',
    'brent',
    'golden',
    'armiho',
    'nester',
    'wolf',
    или пользовательский метод
7. args - аргументы для оракула (X, labels, outers(опционально))
8. search_kwargs - аргументы для метода одномерной оптимизации
9. условие(я) остановки: epsilon, max_iter, max_time, max_oracle


Возвращает:

1. x - результат
2. iterations - range от 0 до количества итераций
3. times - массив времён от начала вычисления до текущего шага
4. values - массив значений функции
5. grad_ratios - массив отношений $||\nabla f(w_k)||^2/||\nabla f(w_0)||^2$
6. fcalls - масиив количества вызовов функции на каждой итерации
7. gcalls - аналогично для градиента
8. hcalls - аналогично для гессиана

В реализации в main.py возвращаются только последние элементы перечисленных массивов

In [14]:
def optimization_task(oracle, start, method='gradient descent', linear_solver='cg', solver_kwargs=dict([]),
                      one_dim_search=None, args=(), search_kwargs=dict([]),
                      epsilon=0, max_iter=float('inf'), max_time=float('inf')):
    iterations, fcalls, gcalls, hcalls, times, values, grad_ratios = [], [], [], [], [], [], []
    start_time, k, fc, gc, hc = time.time(), 0, 0, 0, 0
    x = start

    if method == 'gradient descent':
        ord = 1
        if one_dim_search is None:
            one_dim_search = 'brent'
    elif method == 'newton':
        ord = 2
        if one_dim_search is None:
            one_dim_search = 'unit step'

    f0, g0, _ = oracle(x, *args, order=1)
    fc += 1
    gc += 1
    fk = f0

    if one_dim_search == 'nester':
        L, L0 = 2, 0
        if 'L0' in search_kwargs.keys():
            L0 = 2 * search_kwargs['L0']
            L = L0

    flag = one_dim_search == 'armiho' or one_dim_search == 'wolf' or one_dim_search == 'nester'

    while True:
        f_, gk, hk = oracle(x, *args, order=ord, no_function=flag)
        # в реализации в main.py всегда стоит флаг no_function
        if not flag:
            fk = f_
            fc += 1
        gc += 1
        if ord == 2:
            hc += 1

        if method == 'gradient descent':
            d = -gk
        elif method == 'newton':
            if linear_solver == 'cg':
                d = solveCG(hk, -gk, **solver_kwargs)
            elif linear_solver == 'cholesky':
                d = solveCholesky(hk, -gk, **solver_kwargs)

        ratio = np.dot(gk, gk) / np.dot(g0, g0)
        iterations.append(k)
        k += 1
        fcalls.append(fc)
        gcalls.append(gc)
        hcalls.append(hc)
        times.append(time.time() - start_time)
        values.append(fk)
        grad_ratios.append(ratio)

        if ratio <= epsilon or k > max_iter or times[-1] > max_time:
            break

        fun = lambda alpha: oracle(x + d * alpha, *args)

        if one_dim_search == 'unit step':
            alpha = 1
        elif one_dim_search == 'golden':
            alpha, oracle_counter = golden_search(fun, **search_kwargs)
            fc += oracle_counter
        elif one_dim_search == 'brent':
            solution = opt.minimize_scalar(fun)
            alpha = solution['x']
            fc += solution['nfev']
        elif one_dim_search == 'armiho':
            alpha, fk, oracle_counter = armiho(fun, **search_kwargs, f0=fk, df0=np.dot(gk, d))
            fc += oracle_counter
        elif one_dim_search == 'wolf':
            f_for_wolf = lambda z: oracle(z, *args)
            g_for_wolf = lambda z: oracle(z, *args, order=1, no_function=True)[1]
            solution = opt.line_search(f_for_wolf, g_for_wolf, x, d, **search_kwargs, gfk=gk, old_fval=fk)
            alpha = solution[0]
            if alpha == None:
                alpha = 1
            fc += solution[1]
            gc += solution[2]
            fk = solution[3]
            if fk == None:
                fk = fun(1)
        elif one_dim_search == 'nester':
            L, fk, oracle_counter = nester(fun, fk, gk, L0=max(L / 2, L0))
            alpha = 1 / L
            fc += oracle_counter
        else:
            alpha = one_dim_search(fun, *args)

        x = x + d * alpha

    return x, iterations, times, values, grad_ratios, fcalls, gcalls, hcalls

#### Метод для построения нескольких графиков логарифмов функций

In [15]:
def graph_several(xs, ys, labels, end=None, beg=0, x_l=None, y_l=None, title=None):
    pylab.rcParams['figure.figsize'] = 15, 7
    pylab.subplot(1, 1, 1)
    if end is None:
        end = max([max(x) for x in xs])
    ends = [np.argmin([np.abs(p - end) for p in x]) for x in xs]
    begs = [np.argmin([np.abs(p - beg) for p in x]) for x in xs]
    xs1 = [xs[i][begs[i]:ends[i]+1] for i in range(len(xs))]
    ys1 = [ys[i][begs[i]:ends[i]+1] for i in range(len(xs))]

    def logs(y):
        return [np.log(v) for v in y]

    for i in range(len(xs)):
        pylab.plot(xs1[i], logs(ys1[i]), label=labels[i])

    pylab.title(title)
    pylab.xlabel(x_l)
    pylab.ylabel(y_l)
    pylab.grid()
    pylab.legend()
    pylab.show()

In [16]:
def graph_several2(xs, ys, labels, end1=None, beg1=0, end2=None, beg2=0, x_l=None, y_l=None, title=None):
    pylab.rcParams['figure.figsize'] = 15, 7
    pylab.subplot(1, 2, 1)
    if end1 is None:
        end1 = max([max(x) for x in xs])
    ends = [np.argmin([np.abs(p - end1) for p in x]) for x in xs]
    begs = [np.argmin([np.abs(p - beg1) for p in x]) for x in xs]
    xs1 = [xs[i][begs[i]:ends[i]+1] for i in range(len(xs))]
    ys1 = [ys[i][begs[i]:ends[i]+1] for i in range(len(xs))]

    def logs(y):
        return [np.log(v) for v in y]

    for i in range(len(xs)):
        pylab.plot(xs1[i], logs(ys1[i]), label=labels[i])

    pylab.title(title)
    pylab.xlabel(x_l)
    pylab.ylabel(y_l)
    pylab.grid()
    pylab.legend()
    
    pylab.subplot(1, 2, 2)
    if end2 is None:
        end2 = max([max(x) for x in xs])
    ends = [np.argmin([np.abs(p - end2) for p in x]) for x in xs]
    begs = [np.argmin([np.abs(p - beg2) for p in x]) for x in xs]
    xs1 = [xs[i][begs[i]:ends[i]+1] for i in range(len(xs))]
    ys1 = [ys[i][begs[i]:ends[i]+1] for i in range(len(xs))]

    def logs(y):
        return [np.log(v) for v in y]

    for i in range(len(xs)):
        pylab.plot(xs1[i], logs(ys1[i]), label=labels[i])

    pylab.title(title)
    pylab.xlabel(x_l)
    pylab.ylabel(y_l)
    pylab.grid()
    pylab.legend()
    
    pylab.show()

#### Минимальные значения функций с помощью встроенных методов:

In [17]:
w0_a1a = (2 * np.random.random(X_a1a.shape[1]) - 1) / 2
#начальная точка

w_true_a1a = opt.minimize(function, w0_a1a, args=(X_a1a, labels_a1a), jac = gradient)['x']

f_min_a1a = function(w_true_a1a, X_a1a, labels_a1a)
print('a1a function minimum =', f_min_a1a)

a1a function minimum = 0.29787631704325707


In [18]:
w0_cancer = (2 * np.random.random(X_cancer.shape[1]) - 1) / 2
#начальная точка

w_true_cancer = opt.minimize(function, w0_cancer, args=(X_cancer, labels_cancer), jac = gradient)['x']

f_min_cancer = function(w_true_cancer, X_cancer, labels_cancer)
print('cancer function minimum =', f_min_cancer)

cancer function minimum = 0.07531875761320973


In [19]:
w0_rand = (2 * np.random.random(X_rand.shape[1]) - 1) / 2
#начальная точка


f_min_rand = 0
print('rand function minimum =', f_min_rand)

rand function minimum = 0
